In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import math
from torch.utils import data
from torchvision import transforms
import importlib.util
spec = importlib.util.spec_from_file_location("module.name", "/home/arnab/Desktop/dnn-offloading/Models/bdddataloader.py")
bddloader = importlib.util.module_from_spec(spec)
spec.loader.exec_module(bddloader)


def partition(NN,img,k,CA,f):
    # intializing variables
    A = None
    partition_size = []
    batch = img.shape[0]
    channel = img.shape[1]
    W = [0] * k  # partition
    Windex = []  # partitioning indexes
    init = [0] * (k + 1)
    CA[0] = 0
    CA = [float(val) for val in CA]

    r = img.shape[2]
    c = img.shape[3]
    
    if r > c:
        m = r
    else:
        m = c
    
    
    # sum of capabilities of all nodes
    C2 = np.sum(CA[:(k + 1)])
    
    for i in range(1,k+1):
        C1 = np.sum(CA[:i+1])
        Pi = C1 / C2
        if NN == "conv":
            init[i] = math.floor(Pi * (m - (f - 1)))
            partition_size.append((init[i-1],init[i]+(f-1)))
        elif NN == "ff":
            init[i] = math.floor(Pi * m)
            partition_size.append((init[i - 1],init[i]))

    return partition_size
    
def main():
    # initialize variables
    BATCH_SIZE = 1
    IMAGE_DIM = 227
    kernel = 3
    worker = 2
    CA = [0,4,6]
    
    # load images
    transform = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(IMAGE_DIM),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),])
    
    loader = data.DataLoader(
        bddloader.BDDDataset('/home/arnab/Desktop/Data', train=True, transform=transform),
        batch_size=BATCH_SIZE,
        shuffle=True)
    
    inx = 0
    for img,level in loader:
        print("=> Image : {}".format(inx+1))
        print(img.size())
        after_part = partition("conv",img,worker,CA,kernel)
        print(f"=> Convolutional partition")
        for j in range(len(after_part)):
            print(f"Part {j+1}: {after_part[j]}")
        
        print(f"=> Fully-connected partition")
        after_part = partition("ff",img,worker,CA,0)
        for j in range(len(after_part)):
            print(f"Part {j+1}: {after_part[j]}")
            
        break # one image break
    
if __name__ == '__main__':
    main()




=> Image : 1
torch.Size([1, 3, 227, 227])
=> Convolutional partition
Part 1: (0, 92)
Part 2: (90, 227)
=> Fully-connected partition
Part 1: (0, 90)
Part 2: (90, 227)
